## algorithm design and anlysis-2025 spring  homework 3
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1  

请证明基于比较的排序算法计算复杂度下界为$\Omega(nlogn)$

you can refer a handwritten picture, pleas upload the picture in /fig/xxx.png
answer：
![jupyter](./fig/3-1.jpg)

## 问题 2 

构建二叉搜索树

> 1. 给定一个数组 [48, 33, 49, 47, 42, 46, 32], 构建一个二叉搜索树存储这些数据，请**绘制**所构建的**二叉搜索树**（标明结点的值）。
> 2. 从第1问的二叉搜索树中删除33，请**绘制**删除33后的**二叉搜索树**（标明结点的值）

- 推荐使用graphviz包绘制树的结构

you can refer a handwritten picture, pleas upload the picture in /fig/xxx.png
answer：
![jupyter](./fig/3-2-1.png)
![jupyter](./fig/3-2-2.png)

## 问题 3 

> 下面是九门课程的开始和结束时间:
>
> [(9:00,12:30), (11:00,14:00), (13:00, 14:30), (9:00,10:30),(13:00, 14:30),(14:00,16:30), (15:00,16:30), (15:00,16:30), (9:00,10:30)]
>
> 请使用贪婪算法为这九门课分配教室，要求在同一天内安排这些课，并且每个教室同一时间只能安排一门课。

**请问最少需要几间教室，罗列出每个教室安排的课程**

idea:
1. **问题抽象**
   - 将每门课看作一个区间 \([s_i, e_i)\)。  
   - 目标是给所有区间着色（分配教室），使得同一时间的区间不共用一个颜色（教室），并且使用最少的颜色数。

2. **贪心策略**  
   - **按开始时间升序** 对所有区间排序。  
   - 用一个**小根堆** `heap` 来维护当前已分配各教室的**最早空闲时间**（即各教室上一次课程的结束时间）。

3. **分配流程**  
   
   1. 遍历每个课程区间 (s, e)：
       a. 如果 heap.top() ≤ s：
           → 说明某教室已空闲；弹出 heap.top()，复用该教室。
       b. 否则：
           → 需要开辟一个新教室。
       c. 将 e（该教室的新空闲时间）推入 heap。

In [2]:
# code：
import heapq

def convert_to_minutes(time_str: str) -> int:
    """Convert 'H:M' to minutes."""
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

def convert_to_time_format(minutes: int) -> str:
    """Convert minutes to 'H:M' format."""
    return f"{minutes // 60}:{minutes % 60:02d}"

# Original 9 courses (in the given order)
course_schedules = [
    ("9:00",  "12:30"),
    ("11:00", "14:00"),
    ("13:00", "14:30"),
    ("9:00",  "10:30"),
    ("13:00", "14:30"),
    ("14:00", "16:30"),
    ("15:00", "16:30"),
    ("15:00", "16:30"),
    ("9:00",  "10:30"),
]

# Convert to (start_min, end_min, idx)
course_intervals = [
    (convert_to_minutes(start), convert_to_minutes(end), idx + 1)
    for idx, (start, end) in enumerate(course_schedules)
]
# Sort by start time in ascending order
course_intervals.sort(key=lambda x: x[0])

# Min-heap: stores (current free end time, room_id)
room_heap = []
# Room assignment result: room_id -> list of (course_idx, start, end)
room_allocation = {}

for start, end, idx in course_intervals:
    if room_heap and room_heap[0][0] <= start:
        # Can reuse the earliest free room
        free_end_time, room_id = heapq.heappop(room_heap)
    else:
        # Need to open a new room
        room_id = len(room_allocation) + 1
        room_allocation[room_id] = []
    # Schedule to room_id and update the free time of the room
    room_allocation[room_id].append((idx, start, end))
    heapq.heappush(room_heap, (end, room_id))

# Output results
print(f"Minimum number of rooms needed: {len(room_allocation)}\n")
for room_id in sorted(room_allocation):
    print(f"Room {room_id}:")
    for course_idx, start, end in room_allocation[room_id]:
        print(f"  Course {course_idx}: {convert_to_time_format(start)} - {convert_to_time_format(end)}")
    print()


Minimum number of rooms needed: 3

Room 1:
  Course 1: 9:00 - 12:30
  Course 5: 13:00 - 14:30
  Course 7: 15:00 - 16:30

Room 2:
  Course 4: 9:00 - 10:30
  Course 2: 11:00 - 14:00
  Course 6: 14:00 - 16:30

Room 3:
  Course 9: 9:00 - 10:30
  Course 3: 13:00 - 14:30
  Course 8: 15:00 - 16:30



## 问题 4 

> 爬楼梯问题：假设爬楼梯时你每次只能爬一阶或者爬两阶，问爬上n阶的楼梯，你一共有多少种方法

请设计算法完成该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：
·基本情况：如果楼梯只有 1 阶或 0 阶（即没有楼梯），那么只有一种方法到达顶端，即直接一步跨过（或不需要跨步）。
·状态转移方程：对于 n > 1 的情况，到达第 n 阶的方法数等于到达第 n-1 阶的方法数（从第 n-1 阶跨一步上来）加上到达第 n-2 阶的方法数
 （从第 n-2 阶跨两步上来）。这是因为到达第 n 阶的最后两步只能是从 n-1 跨一步或从 n-2 跨两步。
·迭代计算：使用两个变量 a 和 b 来迭代计算到达每一阶的方法数。a 保存到达第 n-2 阶的方法数，b 保存到达第 n-1 阶的方法数。每次迭代更新
  这两个变量，b 变为 a + b（即到达第 n 阶的方法数），然后 a 更新为 b 的旧值。

In [4]:
# add your code here
def count_ways_to_climb(steps: int) -> int:
    if steps <= 1:
        return 1

    # 使用 prev1 和 prev2 分别存储前两步的结果
    prev1, prev2 = 1, 1
    for _ in range(2, steps + 1):
        current = prev1 + prev2  # 当前步数的方法数
        prev1, prev2 = prev2, current  # 更新前两步的结果

    return prev2

# 测试
for steps in range(6):
    print(f"steps={steps} -> {count_ways_to_climb(steps)} 种方法")

# your algorithm time complexity is: 
#    总时间复杂度为 O(n)。算法需要从第 2 阶迭代到第 n 阶，总共进行了 n-1 次迭代。每次迭代的操作是常数时间的（即更新两个变量的值），
# 因此总的时间复杂度是线性的，与楼梯的阶数 n 成正比。

steps=0 -> 1 种方法
steps=1 -> 1 种方法
steps=2 -> 2 种方法
steps=3 -> 3 种方法
steps=4 -> 5 种方法
steps=5 -> 8 种方法


## 问题 5 

> 0-1背包问题：现在有4块大理石，每个大理石对应的重量和价值使用一个元组表示, 即（重量，价值），4块大理石的重量和价值为：[(5,10), (4,40), (6,30), (3, 50)], 假设你有一辆最大承重为9的小推车，请问使用这个小推车装大理石的最大价值为多少

请设计一个算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：
1. **问题抽象**  
   - 有 \(n\) 件物品，每件物品有重量 \(wt_i\) 和价值 \(val_i\)。  
   - 背包最大承重为 \(W\)，每件物品只能选 0 或 1 次，求最大总价值。

2. **状态定义**  
   - 用一维数组 `dp[w]` 表示当前背包容量恰为 `w` 时能达到的最大价值。

3. **状态转移（倒序遍历）**  
   - 对于第 \(i\) 件物品（`wt=wt_i`, `v=val_i`），  
     ```text
     dp[w] = max(dp[w], dp[w - wt] + v)   for w = W..wt
     ```
   - “倒序”确保每件物品只被使用一次。

4. **初始化**  
   - `dp[0..W]` 初始全 0。




In [6]:
# add your code here
def solve_knapsack(item_weights, item_values, max_capacity):
    """
    解决 0-1 背包问题，计算在给定容量下能获得的最大价值。
    使用动态规划，空间优化版本。
    """
    num_items = len(item_weights)
    # dp[j] 表示容量为 j 时的最大价值
    dp = [0] * (max_capacity + 1)

    for i in range(num_items):
        weight = item_weights[i]
        value = item_values[i]
        # 倒序更新，确保每个物品只被使用一次
        for j in range(max_capacity, weight - 1, -1):
            dp[j] = max(dp[j], dp[j - weight] + value)

    return dp[max_capacity]

# 示例数据
item_weights = [5, 4, 6, 3]
item_values = [10, 40, 30, 50]
max_capacity = 9

# 计算最大价值
optimal_value = solve_knapsack(item_weights, item_values, max_capacity)
print(f"在容量 {max_capacity} 下，最大价值为 {optimal_value}")

# your algorithm time complexity is:时间复杂度为：O(n·W)， n 为物品数，W 为背包容量

在容量 9 下，最大价值为 90


## 问题 6 
给定 `n` 个非负整数表示每个宽度为 `1` 的柱子的高度图，计算按此排列的柱子，下雨之后能接多少雨水。



<div style="text-align:center;width:800px;"> <img alt="" src="./fig/hw3q6.png"> </div>

请分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：  
1. **问题抽象**  
   给定数组 `height`，每个元素表示柱子高度，求柱子间能接多少雨水。

2. **双指针法**  
   - 初始化：`left = 0`，`right = n-1`，`left_max = 0`，`right_max = 0`。  
   - 当 `height[left] < height[right]`：  
     - 若 `height[left] ≥ left_max`，更新 `left_max = height[left]`。  
     - 否则，累加雨水量 `left_max - height[left]`。  
     - 移动 `left += 1`。  
   - 否则：  
     - 若 `height[right] ≥ right_max`，更新 `right_max = height[right]`。  
     - 否则，累加雨水量 `right_max - height[right]`。  
     - 移动 `right -= 1`。  

3. **结束条件**  
   当 `left > right` 时结束，累计值即为结果。


In [11]:
# add your code here
from typing import List

def calculate_trapped_water(elevations: List[int]) -> int:
    """
    使用双指针法计算柱状图中可以接住的雨水总量。
    时间复杂度 O(n)，空间复杂度 O(1)。
    """
    if not elevations:
        return 0

    left_index, right_index = 0, len(elevations) - 1
    max_left = max_right = 0
    total_trapped = 0

    while left_index <= right_index:
        if elevations[left_index] < elevations[right_index]:
            # 左侧柱子较矮，更新左侧最大高度
            if elevations[left_index] >= max_left:
                max_left = elevations[left_index]
            else:
                total_trapped += max_left - elevations[left_index]
            left_index += 1
        else:
            # 右侧柱子较矮或相等，更新右侧最大高度
            if elevations[right_index] >= max_right:
                max_right = elevations[right_index]
            else:
                total_trapped += max_right - elevations[right_index]
            right_index -= 1

    return total_trapped

# 示例测试
if __name__ == "__main__":
    test_cases = [
        ([0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1], 6),
        ([4, 2, 0, 3, 2, 5], 9),
    ]
    for heights, expected_result in test_cases:
        result = calculate_trapped_water(heights)
        print(f"柱状图高度: {heights}\n  -> 接住的雨水量: {result} (预期结果: {expected_result})\n")

# your algorithm time complexity is: O(n)。双指针只向中间移动，每个位置访问一次。


柱状图高度: [0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1]
  -> 接住的雨水量: 6 (预期结果: 6)

柱状图高度: [4, 2, 0, 3, 2, 5]
  -> 接住的雨水量: 9 (预期结果: 9)



## 问题 7 

**股票投资组合优化**： 假设你是一位投资者，想要在不同的股票中分配你的资金，以最大化你的投资回报。每只股票都有不同的预期收益率和风险。你的目标是选择一些股票，使得总投资金额不超过你的预算，并且预期收益最大化。

在这个情况下，你可以将每只股票视为一个“物品”，其重量为投资金额，价值为预期收益率。然后，你可以使用分级背包问题的方法来选择部分股票，以便在预算内获得最大的预期收益。

以下是一个简化的例子：

假设你有以下三只股票可供选择：

1. 股票 A：投资金额 5000 美元，预期收益率 10%
2. 股票 B：投资金额 3000 美元，预期收益率 8%
3. 股票 C：投资金额 2000 美元，预期收益率 12%

请设计算法找到最优投资方案，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：
    这是一个0-1背包问题，其中每个股票可以看作一个物品，其重量为投资金额，价值为预期收益率。
### 算法思路分析

1. **问题抽象**
   - 给定一系列资产（股票），每项资产有三个属性：
     - 名称（`name`）
     - 成本（`cost`）
     - 收益率（`return_rate`）
   - 目标：在给定总预算的情况下，选择一组资产，使得预期的绝对收益最大化。

2. **动态规划求解**
   - **预处理**：计算每项资产的绝对收益值 `value = cost * return_rate`。
   - **状态定义**：`dp[i][j]` 表示在前 `i` 项资产中，使用预算 `j` 时能够获得的最大收益。
   - **状态转移**：
     - 不选择第 `i` 项资产：`dp[i][j] = dp[i-1][j]`。
     - 选择第 `i` 项资产（如果预算足够）：`dp[i][j] = dp[i-1][j - cost_i] + value_i`。
     - 取两者的最大值作为当前状态的值。
   - **初始化**：`dp[0][j] = 0`，表示没有资产时收益为 0。
   - **填表顺序**：从第一项资产开始，逐项更新 `dp` 表。

3. **回溯选择资产**
   - 从 `dp[num_assets][total_budget]` 开始，回溯找出被选中的资产。
   - 如果 `dp[i][j] != dp[i-1][j]`，说明选择了第 `i` 项资产，将其加入结果列表，并更新预算 `j -= cost_i`。
   - 继续回溯直到处理完所有资产。

4. **返回结果**
   - 返回最大预期收益和被选中的资产名称列表。

### 时间复杂度
- **时间复杂度**：\(O(n \times \text{budget})\)，其中 \(n\) 是资产数量，\(\text{budget}\) 是总预算。
- **空间复杂度**：\(O(n \times \text{budget})\)，用于存储动态规划表。

In [22]:
# add your code here
from typing import List, Tuple

def maximize_investment_return(
    assets: List[Tuple[str, int, float]],
    total_budget: int
) -> Tuple[float, List[str]]:
    """
    assets: List of tuples (name, cost, return_rate)
    total_budget: 总投资预算（整数）
    返回: (最大预期绝对收益, 所选资产名称列表)
    """
    num_assets = len(assets)
    # 预先计算每项资产的绝对收益价值 v_i = cost * rate
    asset_costs = [a[1] for a in assets]
    asset_values = [a[1] * a[2] for a in assets]
    asset_names = [a[0] for a in assets]

    # dp[i][j] = 前 i 项资产、预算 j 时的最大收益
    dp = [[0.0] * (total_budget + 1) for _ in range(num_assets + 1)]

    # 动态规划填表
    for i in range(1, num_assets + 1):
        cost_i = asset_costs[i - 1]
        value_i = asset_values[i - 1]
        for j in range(total_budget + 1):
            # 不选择当前资产
            dp[i][j] = dp[i - 1][j]
            # 选择当前资产（如果预算足够）
            if j >= cost_i:
                candidate_value = dp[i - 1][j - cost_i] + value_i
                if candidate_value > dp[i][j]:
                    dp[i][j] = candidate_value

    # 回溯找出被选资产
    chosen_assets = []
    remaining_budget = total_budget
    for i in range(num_assets, 0, -1):
        # 如果当前预算下的最大收益与前一项资产的最大收益不同，则选择了当前资产
        if dp[i][remaining_budget] != dp[i - 1][remaining_budget]:
            chosen_assets.append(asset_names[i - 1])
            remaining_budget -= asset_costs[i - 1]

    chosen_assets.reverse()
    return dp[num_assets][total_budget], chosen_assets

# ---------- 示例 ----------

if __name__ == "__main__":
    assets = [
        ("StockA", 5000, 0.10),  # 投资 5000 美元，预期收益率 10%
        ("StockB", 3000, 0.08),  # 投资 3000 美元，预期收益率 8%
        ("StockC", 2000, 0.12),  # 投资 2000 美元，预期收益率 12%
    ]
    total_budget = 7000  # 总预算 7000 美元

    max_return, selected_assets = maximize_investment_return(assets, total_budget)
    print(f"总预算: {total_budget} 美元")
    print(f"最大预期收益: {max_return:.2f} 美元")
    print("投资组合:", selected_assets)
    
# your algorithm time complexity is:动态规划的时间复杂度为 O(nW)，其中n是股票数量，W 是预算。

总预算: 7000 美元
最大预期收益: 740.00 美元
投资组合: ['StockA', 'StockC']


## 问题 8

给你二叉搜索树的根节点 root ，该树中的恰好两个节点的值被错误地交换。请在不改变其结构的情况下，恢复这棵树 。设计算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

<div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入：root = [1,3,null,null,2]， 输出为：[3,1,null,null,2]， 

解释：3 不能是 1 的左孩子，因为 3 > 1 。交换 1 和 3 使二叉搜索树有效。

2.

 <div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入： root=[3,1,4,null,null,2]， 输出为：[2,1,4,null,null,3]

解释：2 不能在 3 的右子树中，因为 2 < 3 。交换 2 和 3 使二叉搜索树有效。

idea：
### 1. 问题抽象  
给定一棵 BST，由于恰好有两个节点的值被错误地交换，需要在不改变树结构的前提下，将它们的值恢复，使整棵树重新满足 BST 的性质。

### 2. 算法思路（中序遍历 + 双指针）  
中序遍历：对BST进行中序遍历，结果应该是一个递增的序列。通过比较中序遍历的结果与预期的递增序列，我们可以找到两个交换的节点。

找到交换的节点：在中序遍历过程中，记录下不满足BST性质的节点对。

交换节点值：找到两个交换的节点后，交换它们的值。

In [25]:
# add your code here
from typing import List, Union

class TreeNode:
    def __init__(self, value=0, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

def fix_bst(tree_root: TreeNode) -> None:
    """
    修复因两个节点值被交换而失效的二叉搜索树。
    直接修改节点的值，不改变树结构。
    """
    first_node = second_node = previous_node = None

    def inorder_traversal(node):
        nonlocal first_node, second_node, previous_node
        if not node:
            return
        inorder_traversal(node.left)
        if previous_node and previous_node.value > node.value:
            if not first_node:
                first_node, second_node = previous_node, node
            else:
                second_node = node
        previous_node = node
        inorder_traversal(node.right)

    inorder_traversal(tree_root)
    if first_node and second_node:
        first_node.value, second_node.value = second_node.value, first_node.value

def construct_tree(values: List[Union[int, None]]) -> Union[TreeNode, None]:
    """
    根据层序列表（None 表示空节点）构造二叉树并返回根节点。
    """
    if not values:
        return None
    nodes = [None if v is None else TreeNode(v) for v in values]
    child_index = 1
    for i, node in enumerate(nodes):
        if node is not None:
            if child_index < len(nodes):
                node.left = nodes[child_index]
                child_index += 1
            if child_index < len(nodes):
                node.right = nodes[child_index]
                child_index += 1
    return nodes[0]

def convert_tree_to_list(tree_root: Union[TreeNode, None]) -> List[Union[int, None]]:
    """
    将二叉树按层序转成列表，末尾的 None 自动去掉。
    """
    if not tree_root:
        return []
    from collections import deque
    queue = deque([tree_root])
    result = []
    while queue:
        node = queue.popleft()
        if node:
            result.append(node.value)
            queue.append(node.left)
            queue.append(node.right)
        else:
            result.append(None)
    while result and result[-1] is None:
        result.pop()
    return result

# 测试用例
test_cases = [
    # (输入层序列表, 预期修复后的层序列表)
    ([1, 3, None, None, 2], [3, 1, None, None, 2]),
    ([3, 1, 4, None, None, 2], [2, 1, 4, None, None, 3]),
]

for input_values, expected_output in test_cases:
    root_node = construct_tree(input_values)
    print("输入树结构:", input_values)
    fix_bst(root_node)
    output_values = convert_tree_to_list(root_node)
    print("修复后树结构:", output_values)
    print("预期树结构:", expected_output)
    print("测试结果:", "通过" if output_values == expected_output else "失败")
    print("-" * 40)
# your algorithm time complexity is:总的时间复杂度为 O(n)。

输入树结构: [1, 3, None, None, 2]
修复后树结构: [3, 1, None, None, 2]
预期树结构: [3, 1, None, None, 2]
测试结果: 通过
----------------------------------------
输入树结构: [3, 1, 4, None, None, 2]
修复后树结构: [2, 1, 4, None, None, 3]
预期树结构: [2, 1, 4, None, None, 3]
测试结果: 通过
----------------------------------------


## 问题 9  

给你一个二叉树的根节点 root ，树中每个节点都存放有一个 0 到 9 之间的数字。每条从根节点到叶节点的路径都代表一个数字：

> 例如，从根节点到叶节点的路径 1 -> 2 -> 3 表示数字 123 。

设计一个算法计算从根节点到叶节点生成的所有数字之和 ，分析算法设计思路，计算时间复杂度，并基于python编程实现

叶节点:是指没有子节点的节点。

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-1.jpg"> </div>

 输入：root = [1,2,3], 输出：25

解释：从根到叶子节点路径 1->2 代表数字 12，1->3 代表数字 13，因此和为 12 + 13 = 25 。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-2.jpg"> </div>

 输入：root = [4,9,0,5,1], 输出：1026

解释：从根到叶子节点路径 4->9->5 代表数字 495，4->9->1 代表数字 491，4->0 代表数字 40。因此和为 495 + 491 + 40 = 1026 。

idea：
   - 定义递归函数 `dfs(node, cur)` 表示当前遍历到 `node`，`cur` 是从根到父节点已拼出的数值。  
   - 进入 `node` 时，更新当前数值 `cur = cur * 10 + node.val`。  
   - 若 `node` 是叶子节点（`node.left` 和 `node.right` 都为空），则将 `cur` 累加到全局结果中。  
   - 否则，递归遍历 `node.left` 和 `node.right`，将更新后的 `cur` 传下去。

In [27]:
# add your code here
from typing import List, Union

class TreeNode:
    def __init__(self, val: int = 0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def sum_root_to_leaf(root: TreeNode) -> int:
    """
    计算所有从根到叶子的路径数字之和。
    """
    total = 0

    def dfs(node: TreeNode, cur: int):
        nonlocal total
        if not node:
            return
        # 更新当前路径的数值
        cur = cur * 10 + node.val
        # 如果是叶子节点，累加到 total
        if not node.left and not node.right:
            total += cur
            return
        # 继续向下遍历
        dfs(node.left, cur)
        dfs(node.right, cur)

    dfs(root, 0)
    return total

# ---------- 示例测试 ----------

# 辅助函数：按层序创建二叉树
from collections import deque

def build_tree(vals: List[Union[int, None]]) -> Union[TreeNode, None]:
    if not vals:
        return None
    nodes = [None if v is None else TreeNode(v) for v in vals]
    q = deque(nodes)
    root = q.popleft()
    for node in nodes:
        if node:
            if q:
                node.left = q.popleft()
            if q:
                node.right = q.popleft()
    return root

# 示例 1
#    1
#   / \
#  2   3
root1 = TreeNode(1, TreeNode(2), TreeNode(3))
print(sum_root_to_leaf(root1))  # 输出 25 (12 + 13)

# 示例 2
#      4
#     / \
#    9   0
#   / \
#  5   1
root2 = TreeNode(4,
                 TreeNode(9, TreeNode(5), TreeNode(1)),
                 TreeNode(0))
print(sum_root_to_leaf(root2))  # 输出 1026 (495 + 491 + 40)
    
    
# your algorithm time complexity is: **时间复杂度**：O(n)，每个节点访问一次。  


25
1026


## 问题 10  

给你一个二叉树的根节点 root ， 检查它是否轴对称。

1. 分析算法设计思路，计算时间复杂度，并基于python编程实现
2. \* 设计使用递归和迭代两种方法解决这个问题，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-1.png"> </div>

 输入：root = [1,2,2,3,4,4,3]， 输出：true

解释：二叉树 [1,2,2,3,4,4,3] 对称。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-2.png"> </div>

 输入：root = [1,2,2,null,3,null,3]， 输出：false

idea： 设计一个辅助函数 `is_mirror(t1, t2)`：  
  1. 如果 `t1` 和 `t2` 都为空，返回 `True`；  
  2. 如果其中一个为空或 `t1.val != t2.val`，返回 `False`；  
  3. 递归检查：  
     - `t1` 的左子树与 `t2` 的右子树是否镜像  
     - **且** `t1` 的右子树与 `t2` 的左子树是否镜像 

In [28]:
# add your code here
# add your code here：
class TreeNode:
    def __init__(self, val: int=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def is_symmetric_recursive(root: TreeNode) -> bool:
    if not root:
        return True

    def is_mirror(t1: TreeNode, t2: TreeNode) -> bool:
        if not t1 and not t2:
            return True
        if not t1 or not t2 or t1.val != t2.val:
            return False
        # 外侧 & 内侧 同时镜像
        return (is_mirror(t1.left, t2.right)
                and is_mirror(t1.right, t2.left))

    return is_mirror(root.left, root.right)
root1 = TreeNode(1,
             TreeNode(2, TreeNode(3), TreeNode(4)),
             TreeNode(2, TreeNode(4), TreeNode(3)))
print(is_symmetric_recursive(root1))  # True


# 非对称树 [1,2,2,None,3,None,3]
root2 = TreeNode(1,
             TreeNode(2, None, TreeNode(3)),
             TreeNode(2, None, TreeNode(3)))
print(is_symmetric_recursive(root2))  # False

# your algorithm time complexity is:(O(n)


True
False


## 问题 11  

给你一个由 '1'（陆地）和 '0'（水）组成的的二维网格，请你计算网格中岛屿的数量。岛屿总是被水包围，并且每座岛屿只能由水平方向和/或竖直方向上相邻的陆地连接形成。

此外，你可以假设该网格的四条边均被水包围。

分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.
输入：grid = [

    ["1","1","1","1","0"],

    ["1","1","0","1","0"], 

    ["1","1","0","0","0"], 

    ["0","0","0","0","0"]  

]

输出：1

2.
输入：grid = [

    ["1","1","0","0","0"],

    ["1","1","0","0","0"],

    ["0","0","1","0","0"],

    ["0","0","0","1","1"]

]

输出：3

idea:
**Flood Fill（深度优先遍历）**  
   - 遍历网格的每个位置 `(i, j)`：  
     - 若 `grid[i][j] == '1'`，说明发现一个新岛屿，答案加 1；  
     - 调用 `dfs(i, j)` 将该岛屿上的所有 `'1'` 都染成 `'0'`（或访问标记），避免重复计数。  
   - `dfs(i, j)`：  
     1. 边界检查：若越界或 `grid[i][j] != '1'`，直接返回；  
     2. 将 `grid[i][j] = '0'`；  
     3. 递归访问上下左右四个方向。

In [29]:
# add your code here
from typing import List

def num_islands(grid: List[List[str]]) -> int:
 """
 计算网格中岛屿的数量。使用 DFS 对每个新发现的岛屿进行 flood fill。
 时间复杂度 O(mn)，空间复杂度 O(mn)（递归栈）。
 """
 if not grid or not grid[0]:
     return 0

 m, n = len(grid), len(grid[0])
 count = 0

 def dfs(i: int, j: int):
     # 边界或非陆地时返回
     if i < 0 or i >= m or j < 0 or j >= n or grid[i][j] != '1':
         return
     # 将该格标记为已访问（染成水）
     grid[i][j] = '0'
     # 递归四个方向
     dfs(i-1, j)
     dfs(i+1, j)
     dfs(i, j-1)
     dfs(i, j+1)

 for i in range(m):
     for j in range(n):
         if grid[i][j] == '1':
             count += 1
             dfs(i, j)

 return count

# ---------- 示例测试 ----------
if __name__ == "__main__":
 grid1 = [
     ["1","1","1","1","0"],
     ["1","1","0","1","0"],
     ["1","1","0","0","0"],
     ["0","0","0","0","0"]
 ]
 print(num_islands(grid1))  # 输出 1

 grid2 = [
     ["1","1","0","0","0"],
     ["1","1","0","0","0"],
     ["0","0","1","0","0"],
     ["0","0","0","1","1"]
 ]
 print(num_islands(grid2))  # 输出 3
    
    
# your algorithm time complexity is:O(mn)


1
3
